In [1]:
import pandas as pd
import numpy as np
import copy

In [2]:
# broadband and work from home data
broadband_work = pd.read_csv('data/city_broadband_work.csv')
broadband_work.head(5)

,state,city,avg_numproviders,median_numproviders,None of the above / Not working for pay,I continue to commute to work,Used to work from home and still do,"Used to commute, now work from home",I have recently been furloughed or laid-off,"Used to work from home, but now I commute",place_2000_pop
0,AK,Anchorage,NaN,NaN,12.0,7.0,1.0,6.0,1.0,0.0,NaN
1,AK,Wasilla,3.0,3.0,5.0,0.0,0.0,2.0,1.0,0.0,5469
2,AL,Alabaster,4.0,4.0,9.0,6.0,2.0,2.0,1.0,0.0,22619
3,AL,Andalusia,5.0,5.0,3.0,1.0,0.0,1.0,0.0,0.0,8794
4,AL,Atmore,6.0,6.0,3.0,2.0,0.0,1.0,0.0,0.0,7676


In [3]:
# county-year level instrument data
instruments = pd.read_stata('data/broadband_slope_lightning.dta')
instruments['county'] = instruments['FIPS'].astype(int)

In [4]:
# original crosswalk from Missouri data center
xwalk = pd.read_csv('data/place_county.csv')
# get cities and state 2dig abbrv
xwalk = xwalk.loc[xwalk['PlaceName'].str.endswith('city')]
xwalk['city'] = xwalk['PlaceName'].str.split(' city').str[0]
xwalk['state'] = xwalk['CntyName'].str[-2:]
xwalk['county'] = xwalk['county'].astype(int)
xwalk['place_2000_pop'] = xwalk['pop100']
xwalk['placecode'] = xwalk['placefp']
# just keep necessary columns
xwalk = xwalk.loc[:, ['state', 'county','city', 'afact', 'place_2000_pop', 'placecode']]
xwalk.head(5)

,state,county,city,afact,place_2000_pop,placecode
1,AL,1067,Abbeville,1.000,2987,00124
2,AL,1073,Adamsville,1.000,4965,00460
5,AL,1117,Alabaster,1.000,22619,00820
6,AL,1095,Albertville,1.000,17247,00988
7,AL,1123,Alexander City,1.000,15008,01132


In [5]:
# merge crosswalk to instrument data
merged = instruments.merge(xwalk, on = 'county', how = 'left')

# get average lightning and slope at the state and city level
mgby = merged.groupby(['state', 'city'])
final = pd.DataFrame()
final['avglightning'] = mgby['Lightning'].mean()
final['avgslope'] =  mgby['Slope'].mean()

# merge to broadband and work from home data and output to csv
final = final.reset_index()
final = final.merge(broadband_work, on = ['state', 'city'])
final.to_csv('data/instr_broadband_work.csv')